In [4]:
import torch
from models.base_model import ViSynoSenseEmbedding
from transformers import PhobertTokenizerFast
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = PhobertTokenizerFast.from_pretrained("BERT_BASE-20250820T150905Z-1-001/BERT_BASE")
from utils.span_extractor import SpanExtractor

import torch
span_ex =SpanExtractor(tokenizer)
model = ViSynoSenseEmbedding.from_pretrained("BERT_BASE-20250820T150905Z-1-001/BERT_BASE",tokenizer).to(device)

ModuleNotFoundError: No module named 'transformers'

In [ ]:
!git clone --single-branch --branch fast_tokenizers_BARTpho_PhoBERT_BERTweet https://github.com/datquocnguyen/transformers.git
!pip install -e transformers


Cloning into 'transformers'...
